## Tips and tricks for drawing insights from Google Analytics data in BQ

**The GA4's export lacks a session-level table.** See [here](https://blog.datatovalue.nl/ga4-bigquery-sql-how-to-build-queries-from-scratch-55869cda290a).

We need the following dimensions:
- date: obviously to determine the date of the session
- user_pseudo_id + session ID: to differentiate between each user and session, can be used for getting the ‘users’ or ‘sessions’ metric (with count(distinct session_id))
- device_category: to get the device breakdown
- acquisition channel: can be as simple as just the session source/medium or some sort of channel grouping, used for the channel breakdown

Other useful sources:
- https://ga4bigquery.com/
- https://tanelytics.com/category/google-cloud/bigquery/
- https://adswerve.com/blog/author/luka-cempre
- https://gtm-gear.com/

In [1]:
%load_ext google.cloud.bigquery

In [5]:
%%bigquery
with prep as (
select
  cast(event_date as date format 'YYYYMMDD') as date,
  user_pseudo_id,
  concat(user_pseudo_id, "_", (select value.int_value from unnest(event_params) where key = 'ga_session_id')) as session_id,
  event_timestamp,
  device.category as device_category,
  (select
        as struct
        (select value.string_value from unnest(event_params) where key = 'source') as source_value,
        (select value.string_value from unnest(event_params) where key = 'medium') as medium,
        (select value.string_value from unnest(event_params) where key = 'campaign') as campaign,
        (select value.string_value from unnest(event_params) where key = 'gclid') as gclid,
    ) as traffic_source
from
  `johannesvc.analytics_413581908.events_*`
-- where
--     contains_substr(_table_suffix, 'intraday') is not true 
    -- and cast(_table_suffix as date format 'YYYYMMDD') = '2024-02-12'
),

prep2 as (
select date,
       user_pseudo_id,
       session_id,
       device_category,
       array_agg(
        if(coalesce(traffic_source.source_value, traffic_source.medium,traffic_source.gclid) is not null, traffic_source, null)
        order by event_timestamp asc limit 1
        )[safe_offset(0)] as session_first_traffic_source
from prep
group by 1,2,3,4
)

select 
    date,
    user_pseudo_id,
    session_id,
    device_category,
    lower(if(session_first_traffic_source.gclid is not null,'google',ifnull(session_first_traffic_source.source_value,'direct'))) as session_source,
    lower(if(session_first_traffic_source.gclid is not null,'cpc',ifnull(session_first_traffic_source.medium,'(none)'))) as session_medium,
    session_first_traffic_source.campaign as session_campaign
from prep2
ORDER BY date;

Query is running:   0%|          |

Downloading:   0%|          |

,date,user_pseudo_id,session_id,device_category,session_source,session_medium,session_campaign
0,2024-02-15,1416806056.1698414500,1416806056.1698414500_1708027018,mobile,direct,(none),None
1,2024-02-15,1416806056.1698414500,1416806056.1698414500_1708031953,mobile,direct,(none),None
2,2024-02-15,1007198316.1707993467,1007198316.1707993467_1707993466,desktop,direct,(none),None
3,2024-02-17,1505607843.1708194715,1505607843.1708194715_1708194714,mobile,m.facebook.com,referral,(referral)
4,2024-02-17,49639856.1698414481,49639856.1698414481_1708161204,mobile,direct,(none),None
5,2024-02-19,1170469778.1705672363,1170469778.1705672363_1708337867,desktop,direct,(none),None
6,2024-02-20,1170469778.1705672363,1170469778.1705672363_1708442095,mobile,direct,(none),None
7,2024-02-20,1170469778.1705672363,1170469778.1705672363_1708448392,mobile,direct,(none),None
8,2024-02-20,1170469778.1705672363,1170469778.1705672363_1708448392,desktop,direct,(none),None
9,2024-02-20,1170469778.1705672363,1170469778.1705672363_1708442095,desktop,direct,(none),None
